#### ADS 509
#### Assignment 1
#### Abby Tan
#### GitHub Link: https://github.com/Abby-Tan/ADS509_Assignment_1

---

# ADS 509 Module 1: APIs and Web Scraping

This notebook has three parts. In the first part you will pull data from the Twitter API. In the second, you will scrape lyrics from AZLyrics.com. In the last part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 100,000 Twitter followers and 20 songs with lyrics on AZLyrics.com. In this part of the assignment we pull the some of the user information for the followers of your artist and store them in text files. 


## Important Note

This assignment requires you to have a version of Tweepy that is at least version 4. The latest version is 4.10 as I write this. Critically, this version of Tweepy is *not* on the upgrade path from Version 3, so you will not be able to simply upgrade the package if you are on Version 3. Instead you will need to explicitly install version 4, which you can do with a command like this: `pip install "tweepy>=4"`. You will also be using Version 2 of the Twitter API for this assignment. 

Run the below cell. If your version of Tweepy begins with a "4", then you should be good to go. If it begins with a "3" then run the following command, found [here](https://stackoverflow.com/questions/5226311/installing-specific-package-version-with-pip), at the command line or in a cell: `pip install -Iv tweepy==4.9`. (You may want to update that version number if Tweepy has moved on past 4.9. 

In [1]:
pip install tweepy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show tweepy

Name: tweepy
Version: 4.10.1
Summary: Twitter library for Python
Home-page: https://www.tweepy.org/
Author: Joshua Roesslein
Author-email: tweepy@googlegroups.com
License: MIT
Location: c:\users\abby0\anaconda3\lib\site-packages
Requires: oauthlib, requests-oauthlib, requests
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Twitter API Pull

In [1]:
# for the twitter section
import tweepy
import os
import datetime
import re
from pprint import pprint

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter

In [2]:
# Use this cell for any import statements you add
import random
import pandas as pd

We need bring in our API keys. Since API keys should be kept secret, we'll keep them in a file called `api_keys.py`. This file should be stored in the directory where you store this notebook. The example file is provided for you on Blackboard. The example has API keys that are _not_ functional, so you'll need to get Twitter credentials and replace the placeholder keys. 

In [3]:
from api_keys import api_key, api_key_secret, bearer_token

In [4]:
client = tweepy.Client(bearer_token,wait_on_rate_limit=True)

# Testing the API

The Twitter APIs are quite rich. Let's play around with some of the features before we dive into this section of the assignment. For our testing, it's convenient to have a small data set to play with. We will seed the code with the handle of John Chandler, one of the instructors in this course. His handle is `@37chandler`. Feel free to use a different handle if you would like to look at someone else's data. 

We will write code to explore a few aspects of the API: 

1. Pull some of the followers @37chandler.
1. Explore response data, which gives us information about Twitter users. 
1. Pull the last few tweets by @37chandler.


In [144]:
handle = "37chandler"
user_obj = client.get_user(username=handle)

followers = client.get_users_followers(
    # Learn about user fields here: 
    # https://developer.twitter.com/en/docs/twitter-api/data-dictionary/object-model/user
    user_obj.data.id, user_fields=["created_at","description","location",
                                   "public_metrics"]
)

Now let's explore these a bit. We'll start by printing out names, locations, following count, and followers count for these users. 

In [37]:
num_to_print = 20

for idx, user in enumerate(followers.data) :
    following_count = user.public_metrics['following_count']
    followers_count = user.public_metrics['followers_count']
    
    print(f"{user.name} lists '{user.location}' as their location.")
    print(f" Following: {following_count}, Followers: {followers_count}.")
    print()
    
    if idx >= (num_to_print - 1) :
        break

Dave Renn lists 'None' as their location.
 Following: 42, Followers: 10.

Lionel lists 'None' as their location.
 Following: 202, Followers: 204.

Megan Randall lists 'None' as their location.
 Following: 141, Followers: 100.

Jacob Salzman lists 'None' as their location.
 Following: 562, Followers: 134.

twiter not fun lists 'None' as their location.
 Following: 221, Followers: 21.

Hariettwilsonincarnate lists 'None' as their location.
 Following: 218, Followers: 60.

Christian Tinsley lists 'None' as their location.
 Following: 2, Followers: 0.

Steve lists 'I'm over here.' as their location.
 Following: 1590, Followers: 33.

John O'Connor 🇺🇦 lists 'None' as their location.
 Following: 8, Followers: 1.

CodeGrade lists 'Amsterdam' as their location.
 Following: 2820, Followers: 423.

Cleverhood lists 'Providence, RI' as their location.
 Following: 2792, Followers: 3559.

Regina 🚶‍♀️🚲🌳 lists 'Minneapolis' as their location.
 Following: 2796, Followers: 3322.

Eric Hallstrom lists 'Mi

Let's find the person who follows this handle who has the most followers. 

In [38]:
max_followers = 0

for idx, user in enumerate(followers.data) :
    followers_count = user.public_metrics['followers_count']
    
    if followers_count > max_followers :
        max_followers = followers_count
        max_follower_user = user

        
print(max_follower_user)
print(max_follower_user.public_metrics)

WedgeLIVE
{'followers_count': 14169, 'following_count': 2223, 'tweet_count': 56086, 'listed_count': 218}


Let's pull some more user fields and take a look at them. The fields can be specified in the `user_fields` argument. 

In [39]:
response = client.get_user(id=user_obj.data.id,
                          user_fields=["created_at","description","location",
                                       "entities","name","pinned_tweet_id","profile_image_url",
                                       "verified","public_metrics"])

In [40]:
for field, value in response.data.items() :
    print(f"for {field} we have {value}")

for name we have John Chandler
for verified we have False
for created_at we have 2009-04-18 22:08:22+00:00
for description we have He/Him. Data scientist, urban cyclist, educator, erstwhile frisbee player. 

¯\_(ツ)_/¯
for profile_image_url we have https://pbs.twimg.com/profile_images/2680483898/b30ae76f909352dbae5e371fb1c27454_normal.png
for username we have 37chandler
for public_metrics we have {'followers_count': 193, 'following_count': 589, 'tweet_count': 996, 'listed_count': 3}
for id we have 33029025
for location we have MN


Now a few questions for you about the user object.

Q: How many fields are being returned in the `response` object? 

A: 9 fields

---

Q: Are any of the fields within the user object non-scalar? (I.e., more complicated than a simple data type like integer, float, string, boolean, etc.) 

A: Yes. The profile_image_url field is an url link for an image, and the public metrics is a dictionary.

---

Q: How many friends, followers, and tweets does this user have? 

A: This user has 589 friends, 193 followers, and 994 tweets.


Although you won't need it for this assignment, individual tweets can be a rich source of text-based data. To illustrate the concepts, let's look at the last few tweets for this user. You are encouraged to explore the fields that are available about Tweets.

In [41]:
response = client.get_users_tweets(user_obj.data.id)

# By default, only the ID and text fields of each Tweet will be returned
for idx, tweet in enumerate(response.data) :
    print(tweet.id)
    print(tweet.text)
    print()
    
    if idx > 10 :
        break

1569155273742327811
As a Minneapolis person, I knew we had Toronto beat, but I didn't realize Portland had us beat: https://t.co/xrx5mOFcWK.

But @nytimes, c'mon! https://t.co/M9mBWhdgsj

1568982292923826176
RT @wonderofscience: Amazing lenticular cloud over Mount Fuji

Credit: Iurie Belegurschi
https://t.co/0mUxl28H9U

1568242374085869570
RT @depthsofwiki: lots of memes about speedy wikipedia editors — quick thread about what went down on wikipedia in the minutes after her de…

1568074978754703361
@DrLaurenWilson @leighradwood @MaritsaGeorgiou @Walgreens I could not possibly agree more with this sentiment. Compared to almost any other primary care I've received, they are great.

1567530169686196224
@DrLaurenWilson @MaritsaGeorgiou @Walgreens For those who have access to Curry Health Center on campus, you can get a bivalent booster in 15 minutes from their delightful staff.

1567511181526708224
RT @shes_the_maNN1: I can’t describe how ancient this makes me feel. https://t.co/a1IvELjOFY

## Pulling Follower Information

In this next section of the assignment, we will pull information about the followers of your two artists. We've seen above how to pull a set of followers using `client.get_users_followers`. This function has a parameter, `max_results`, that we can use to change the number of followers that we pull. Unfortunately, we can only pull 1000 followers at a time, which means we will need to handle the _pagination_ of our results. 

The return object has the `.data` field, where the results will be found. It also has `.meta`, which we use to select the next "page" in the results using the `next_token` result. I will illustrate the ideas using our user from above. 


### Rate Limiting

Twitter limits the rates at which we can pull data, as detailed in [this guide](https://developer.twitter.com/en/docs/twitter-api/rate-limits). We can make 15 user requests per 15 minutes, meaning that we can pull $4 \cdot 15 \cdot 1000 = 60000$ users per hour. I illustrate the handling of rate limiting below, though whether or not you hit that part of the code depends on your value of `handle`.  


In the below example, I'll pull all the followers, 25 at a time. (We're using 25 to illustrate the idea; when you do this set the value to 1000.) 

In [42]:
handle_followers = []
pulls = 0
max_pulls = 100
next_token = None

while True :

    followers = client.get_users_followers(
        user_obj.data.id, 
        max_results=1000, # when you do this for real, set this to 1000!
        pagination_token = next_token,
        user_fields=["created_at","description","location",
                     "entities","name","pinned_tweet_id","profile_image_url",
                     "verified","public_metrics"]
    )
    pulls += 1
    
    for follower in followers.data : 
        follower_row = (follower.id,follower.name,follower.created_at,follower.description)
        handle_followers.append(follower_row)
    
    if 'next_token' in followers.meta and pulls < max_pulls :
        next_token = followers.meta['next_token']
    else : 
        break

## Pulling Twitter Data for Your Artists

Now let's take a look at your artists and see how long it is going to take to pull all their followers. 

In [43]:
artists = dict()

for handle in ['PostMalone','sushitrash'] : 
    user_obj = client.get_user(username=handle,user_fields=["public_metrics"])
    artists[handle] = (user_obj.data.id, 
                       handle,
                       user_obj.data.public_metrics['followers_count'])
    

for artist, data in artists.items() : 
    print(f"It would take {data[2]/(1000*15*4):.2f} hours to pull all {data[2]} followers for {artist}. ")

It would take 115.87 hours to pull all 6951924 followers for PostMalone. 
It would take 29.48 hours to pull all 1768763 followers for sushitrash. 


Depending on what you see in the display above, you may want to limit how many followers you pull. It'd be great to get at least 200,000 per artist. 

As we pull data for each artist we will write their data to a folder called "twitter", so we will make that folder if needed.

In [44]:
# Make the "twitter" folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then "unlink" it. Then create a new one.

if not os.path.isdir("twitter") : 
    #shutil.rmtree("twitter/")
    os.mkdir("twitter")

In this following cells, build on the above code to pull some of the followers and their data for your two artists. As you pull the data, write the follower ids to a file called `[artist name]_followers.txt` in the "twitter" folder. For instance, for Cher I would create a file named `cher_followers.txt`. As you pull the data, also store it in an object like a list or a data frame.

In addition to creating a file that only has follower IDs in it, you will create a file that includes user data. From the response object please extract and store the following fields: 

* screen_name	
* name	
* id	
* location	
* followers_count	
* friends_count	
* description

Store the fields with one user per row in a tab-delimited text file with the name `[artist name]_follower_data.txt`. For instance, for Cher I would create a file named `cher_follower_data.txt`. 

One note: the user's description can have tabs or returns in it, so make sure to clean those out of the description before writing them to the file. I've included some example code to do that below the stub. 

In [45]:
num_followers_to_pull = 100*1000 # feel free to use this to limit the number of followers you pull.

In [17]:
handles = ['PostMalone','sushitrash']
handle_followers = []    

max_pulls = 101
next_token = None

whitespace_pattern = re.compile(r"\s+")

user_data = dict() 
followers_data = dict()

for handle in handles :
    user_data[handle] = [] # will be a list of lists
    followers_data[handle] = [] # will be a simple list of IDs
    user_obj = client.get_user(username=handle)
    
# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

for handle in handles :
    
    pulls = 0
    
# Create the output file names 
    followers_output_file = handle + "_followers.txt"
    user_data_output_file = handle + "_follower_data.txt"

# Use tweepy.Paginator to connect
    for response in tweepy.Paginator(
                                    client.get_users_followers,
                                    user_obj.data.id,
                                    max_results=1000,
                                    user_fields=["username","name","id","location","description","public_metrics"],
                                    limit=101 ):
        
        print(pulls+1, "pulls completed for", handle)
        pulls += 1

# Format data
        for follower in response.data:
            follower_row = (str(follower.username), str(follower.name), str(follower.id), str(follower.location), 
                        str(follower.public_metrics['followers_count']),
                        str(follower.public_metrics['following_count']),
                        re.sub(r'\s+', ' ', follower.description).strip() )
            followers_data[handle].append(follower_row)
            
            follower_id = str(follower.id)
            user_data[handle].append(follower_id)
            
        if 'next_token' in response.meta and pulls < max_pulls :
            next_token = response.meta['next_token']
        else : 
            break

# Export data
    pd.DataFrame(followers_data[handle]).head(100000).to_csv(os.getcwd() + '\\twitter\\' 
                                                             + followers_output_file, index=False, sep='\t')

    pd.DataFrame(user_data[handle]).head(100000).to_csv(os.getcwd() + '\\twitter\\' 
                                                        + user_data_output_file, index=False, sep='\t')
    
end_time = datetime.datetime.now()
print(end_time - start_time)

1 pulls completed for PostMalone
2 pulls completed for PostMalone
3 pulls completed for PostMalone
4 pulls completed for PostMalone
5 pulls completed for PostMalone
6 pulls completed for PostMalone
7 pulls completed for PostMalone
8 pulls completed for PostMalone
9 pulls completed for PostMalone
10 pulls completed for PostMalone
11 pulls completed for PostMalone
12 pulls completed for PostMalone
13 pulls completed for PostMalone
14 pulls completed for PostMalone


Rate limit exceeded. Sleeping for 893 seconds.


15 pulls completed for PostMalone
16 pulls completed for PostMalone
17 pulls completed for PostMalone
18 pulls completed for PostMalone
19 pulls completed for PostMalone
20 pulls completed for PostMalone
21 pulls completed for PostMalone
22 pulls completed for PostMalone
23 pulls completed for PostMalone
24 pulls completed for PostMalone
25 pulls completed for PostMalone
26 pulls completed for PostMalone
27 pulls completed for PostMalone
28 pulls completed for PostMalone
29 pulls completed for PostMalone


Rate limit exceeded. Sleeping for 892 seconds.


30 pulls completed for PostMalone
31 pulls completed for PostMalone
32 pulls completed for PostMalone
33 pulls completed for PostMalone
34 pulls completed for PostMalone
35 pulls completed for PostMalone
36 pulls completed for PostMalone
37 pulls completed for PostMalone
38 pulls completed for PostMalone
39 pulls completed for PostMalone
40 pulls completed for PostMalone
41 pulls completed for PostMalone
42 pulls completed for PostMalone
43 pulls completed for PostMalone
44 pulls completed for PostMalone


...

64 pulls completed for sushitrash
65 pulls completed for sushitrash
66 pulls completed for sushitrash
67 pulls completed for sushitrash
68 pulls completed for sushitrash
69 pulls completed for sushitrash
70 pulls completed for sushitrash
71 pulls completed for sushitrash
72 pulls completed for sushitrash
73 pulls completed for sushitrash
74 pulls completed for sushitrash
75 pulls completed for sushitrash
76 pulls completed for sushitrash
77 pulls completed for sushitrash
78 pulls completed for sushitrash


Rate limit exceeded. Sleeping for 894 seconds.


79 pulls completed for sushitrash
80 pulls completed for sushitrash
81 pulls completed for sushitrash
82 pulls completed for sushitrash
83 pulls completed for sushitrash
84 pulls completed for sushitrash
85 pulls completed for sushitrash
86 pulls completed for sushitrash
87 pulls completed for sushitrash
88 pulls completed for sushitrash
89 pulls completed for sushitrash
90 pulls completed for sushitrash
91 pulls completed for sushitrash
92 pulls completed for sushitrash
93 pulls completed for sushitrash


Rate limit exceeded. Sleeping for 893 seconds.


94 pulls completed for sushitrash
95 pulls completed for sushitrash
96 pulls completed for sushitrash
97 pulls completed for sushitrash
98 pulls completed for sushitrash
99 pulls completed for sushitrash
100 pulls completed for sushitrash
101 pulls completed for sushitrash
3:15:35.448754


In [67]:
tricky_description = """
    Home by Warsan Shire
    
    no one leaves home unless
    home is the mouth of a shark. 
    you only run for the border
    when you see the whole city
    running as well.

"""
# This won't work in a tab-delimited text file.

clean_description = re.sub(r"\s+"," ",tricky_description).strip()
clean_description

'Home by Warsan Shire no one leaves home unless home is the mouth of a shark. you only run for the border when you see the whole city running as well.'

---

# Lyrics Scrape

This section asks you to pull data from the Twitter API and scrape www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [18]:
artists = {'robyn':"https://www.azlyrics.com/r/robyn.html",
           'cher':"https://www.azlyrics.com/c/cher.html"} 
# we'll use this dictionary to hold both the artist name and the link on AZlyrics

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: According to the limitations of the robots.txt file, the scraping we are about to do is disallow by this page.


In [19]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)


for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    soup = BeautifulSoup(r.text, 'html.parser')
    time.sleep(5 + 10*random.random())

    url = 'https://www.azlyrics.com/'
    lyrics_pages[artist] = [url + a['href'].strip('..') for a in soup.find(id='listAlbum').findAll('a', href=True)]
    
    # Reference: https://stackoverflow.com/questions/61101772/scraping-a-webpage-and-related-subsequent-page-using-r-or-python

Let's make sure we have enough lyrics pages to scrape. 

In [20]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [21]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For robyn we have 104.
The full pull will take for this artist will take 0.29 hours.
For cher we have 318.
The full pull will take for this artist will take 0.88 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [22]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)

In [23]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

Define the two artists that I pick

In [24]:
artists = {'PostMalone':"https://www.azlyrics.com/p/postmalone.html",
           'Joji':"https://www.azlyrics.com/j/joji.html"} 

In [25]:
# Let's set up a dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    soup = BeautifulSoup(r.text, 'html.parser')
    time.sleep(5 + 10*random.random())

    url = 'https://www.azlyrics.com/'
    lyrics_pages[artist] = [url + a['href'].strip('..') for a in soup.find(id='listAlbum').findAll('a', href=True)]
    
# Reference: https://stackoverflow.com/questions/61101772/scraping-a-webpage-and-related-subsequent-page-using-r-or-python

In [26]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [27]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For PostMalone we have 103.
The full pull will take for this artist will take 0.29 hours.
For Joji we have 64.
The full pull will take for this artist will take 0.18 hours.


Get Lyrics

In [28]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

for artist in lyrics_pages:
    # Use this space to carry out the following steps: 
    # 1. Build a subfolder for the artist
    folders = os.getcwd() + '\\lyrics\\' + artist
    os.makedirs(folders)
    total_pages = 0
    # 2. Iterate over the lyrics pages
    for link in lyrics_pages[artist]:
    # 3. Request the lyrics page. 
        # Don't forget to add a line like `time.sleep(5 + 10*random.random())`
        # to sleep after making the request
        r = requests.get(link)
        soup = BeautifulSoup(r.content, 'lxml')
        time.sleep(5 + 10*random.random())
    # 4. Extract the title and lyrics from the page.
        soup = BeautifulSoup(r.content, 'lxml')
        title = re.search('lyrics/(.*).html', link).group(1)
        title = re.findall(r'/(\w+)',title)
        lyric = soup.select_one(".ringtone ~ div").get_text(strip=True, separator="\n")
    # 5. Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_url`
    #    to generate the filename.
        file_name = generate_filename_from_link(link)
        output_lyrics = os.path.join(folders, file_name)
        with open(output_lyrics, 'w') as f:
            f.write('%s\n\n%s' % (title, lyric))       
    # Remember to pull at least 20 songs per artist. It may be fun to pull all the songs for the artist
        total_pages += 1

        if total_pages > 19:
            break    

print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")


# Reference:
# https://www.geeksforgeeks.org/python-regex-re-search-vs-re-findall/
# https://stackoverflow.com/questions/67992715/how-do-i-scrape-lyrics-from-a-list-of-song-lyric-urls

Total run time was 0.12 hours.


---

# Evaluation

This assignment asks you to pull data from the Twitter API and scrape www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [30]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

---

## Checking Twitter Data

The output from your Twitter API pull should be two files per artist, stored in files with formats like `cher_followers.txt` (a list of all follower IDs you pulled) and `cher_followers_data.txt`. These files should be in a folder named `twitter` within the repository directory. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [21]:
twitter_files = os.listdir("twitter")
twitter_files = [f for f in twitter_files if f != ".DS_Store"]
artist_handles = list(set([name.split("_")[0] for name in twitter_files]))

print(f"We see two artist handles: {artist_handles[0]} and {artist_handles[1]}.")

We see two artist handles: PostMalone and sushitrash.


In [121]:
for artist in artist_handles :
    follower_file = artist + "_followers.txt"
    follower_data_file = artist + "_follower_data.txt"
    
    ids = open(os.getcwd() + "\\twitter\\" + follower_file,'r', encoding='utf-8').readlines()
    
    print(f"We see {len(ids)-1} in your follower file for {artist}, assuming a header row.")
    
    with open(os.getcwd() + "\\twitter\\" + follower_data_file,'r' , encoding='utf-8') as infile :
        
        # check the headers
        headers = infile.readline().split("\t")
        
        print(f"In the follower data file ({follower_data_file}) for {artist}, we have these columns:")
        print(" : ".join(headers))
        
        description_words = []
        locations = set()
        
        
        for idx, line in enumerate(infile.readlines()) :
            line = line.strip("\n").split("\t")
            
            try : 
                locations.add(line[3])            
                description_words.extend(line[6])
            except :
                pass
        

        print(f"We have {idx+1} data rows for {artist} in the follower data file.")

        print(f"For {artist} we have {len(locations)} unique locations.")                                    

        print(f"For {artist} we have {len(description_words)} words in the descriptions.")
        print("Here are the five most common words:")
        print(Counter(description_words).most_common(5))

        
        print("")
        print("-"*40)
        print("")
    

We see 100000 in your follower file for PostMalone, assuming a header row.
In the follower data file (PostMalone_follower_data.txt) for PostMalone, we have these columns:
username : name : id : location : followers_count : following_count : description

We have 100230 data rows for PostMalone in the follower data file.
For PostMalone we have 18921 unique locations.
For PostMalone we have 2247554 words in the descriptions.
Here are the five most common words:
[(' ', 338558), ('e', 163180), ('a', 132490), ('o', 120714), ('t', 111625)]

----------------------------------------

We see 100000 in your follower file for sushitrash, assuming a header row.
In the follower data file (sushitrash_follower_data.txt) for sushitrash, we have these columns:
username : name : id : location : followers_count : following_count : description

We have 100242 data rows for sushitrash in the follower data file.
For sushitrash we have 18918 unique locations.
For sushitrash we have 2247727 words in the descri

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [31]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For Joji we have 20 files.
For Joji we have roughly 3787 words, 558 are unique.
For PostMalone we have 20 files.
For PostMalone we have roughly 8757 words, 1174 are unique.


---